In [30]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
%matplotlib qt
import matplotlib.dates as mdates

In [31]:
import os
import pandas as pd

# Participant ID and signals data
participant_id = '11'
signals = pd.read_pickle('/home/art/SignalProcessing/Sessions/P011/labeled_features.pkl')


In [ ]:
output_dir = '/home/art/SignalProcessing/Sessions/Summary'
os.makedirs(output_dir, exist_ok=True)

summary_stats = {'participant_id': participant_id}

all_stats = signals.describe().T[['mean']]  # only mean -AJS
for signal in all_stats.index:
    summary_stats[f'all_{signal}_mean'] = all_stats.loc[signal, 'mean']

for event in signals['event'].unique():
    df_event = signals[signals['event'] == event]
    event_stats = df_event.describe().T[['mean']] # only mean -AJS
    for signal in event_stats.index:
        summary_stats[f'{event}_{signal}_mean'] = event_stats.loc[signal, 'mean']

summary_stats_df = pd.DataFrame([summary_stats])


output_file = os.path.join(output_dir, 'summary_statistics_comparison.csv')

if os.path.exists(output_file):

    existing_data = pd.read_csv(output_file)
    combined_data = pd.concat([existing_data, summary_stats_df], ignore_index=True)
    combined_data.to_csv(output_file, index=False)
else:
    summary_stats_df.to_csv(output_file, index=False)

print(f"Summary statistics for Participant {participant_id} saved to {output_file}")

Summary statistics for Participant 9 saved to /home/art/SignalProcessing/Sessions/Summary/summary_statistics_comparison.csv


In [35]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit_transform(signals['EDA_Phasic'].reshape(-1, 1)).flatten()
#rmssd_eda_phasic = signals['EDA_Phasic'] ** 2
import neurokit2 as nk
signals['EDA_Phasic'] = nk.standardize(signals['EDA_Phasic'])
signals['EDA_Tonic'] = nk.standardize(signals['EDA_Tonic'])

In [ ]:
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd

import matplotlib.dates as mdates

signals.dropna(inplace=True)

signals['EDA_Phasic'] = nk.standardize(signals['EDA_Phasic'], robust=True)
signals['EDA_Tonic'] = nk.standardize(signals['EDA_Tonic'], robust=True)
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_columns = scaler.fit_transform(signals[['EDA_Tonic', 'EDA_Phasic', 'hrv_lf_hf_ratio', 'hrv_hf', 'hrv_rmssd']])

scaled_df = pd.DataFrame(scaled_columns, columns=['EDA_Tonic', 'EDA_Phasic', 'hrv_lf_hf_ratio', 'hrv_hf', 'hrv_rmssd'], index=signals.index)

signals[['EDA_Tonic', 'EDA_Phasic', 'hrv_lf_hf_ratio', 'hrv_hf', 'hrv_rmssd']] = scaled_df


colors = mpl.colormaps['Dark2'].colors
signal_list = [
    'hrv_hf', 'hrv_mean_hr', 'hrv_lf_hf_ratio',
    'hrv_rmssd', 'Motion E4', 'Hexoskin Motion',
    'RSP_Rate', 'EDA_Tonic', 'EDA_Phasic'
]
signals_to_plot = [sig for sig in signals.columns if sig in signal_list]
motion_signals = ["Motion E4", "Hexoskin Motion"]

num_rows = len(signals_to_plot) - len(motion_signals) + 1
fig, axes = plt.subplots(num_rows, 1, figsize=(18, 3 * num_rows), sharex=True)
axes = axes.flatten() if num_rows > 1 else [axes]

row_idx = 0
motion_trace_added = False

signal_labels = {
    'hrv_hf': 'HRV HF',
    'hrv_mean_hr': 'Mean Heart Rate',
    'hrv_lf_hf_ratio': 'HRV LF/HF Ratio',
    'hrv_rmssd': 'HRV RMSSD',
    'Motion E4': 'E4 Motion',
    'Hexoskin Motion': 'Hexoskin Motion',
    'RSP_Rate': 'Respiratory Rate (bpm)',
    'EDA_Tonic': 'EDA (Tonic)',
    'EDA_Phasic': 'EDA (Phasic)'
}

event_highlights = {
    "survey":      ("white", 0.0),
    "cycle_cal":   ("grey", 0.0),
    "matb_train":  ("grey", 0.4),
    "cycle":       ("purple", 0.4),
    "rest_1":      ("white", 0.0),
    "baseline":    ("turquoise", 0.5),
    "trial_1":     ("green", 0.4),
    "rest_2":      ("white", 0.0),
    "trial_2":     ("red", 0.4),
    "recovery":    ("white", 0.0),
}

y_limits = {
    'hrv_hf': (0, 1.2),
    'hrv_mean_hr': (50, 140),
    'hrv_lf_hf_ratio': (0, 1.2),
    'hrv_rmssd': (0, 1.2),
    'Motion E4': (0, 2),
    'Hexoskin Motion': (0, 2),
    'RSP_Rate': (2, 54),
    'EDA_Tonic': (0, 1.2),
    'EDA_Phasic': (0, 1.2)
}

event_labels = {
    "survey":      "Survey",
    "cycle_cal":   "Calibration",
    "matb_train":  "Training",
    "cycle":       "Cycling",
    "rest_1":      "Rest 1",
    "baseline":    "Baseline",
    "trial_1":     "Trial 1",
    "rest_2":      "Resting 2",
    "trial_2":     "Trial 2",
    "recovery":    "Recovery"
}


motion_offset = 0.7
for idx, signal in enumerate(signals_to_plot):
    color = colors[idx % len(colors)]
    ax = axes[row_idx]
    label = signal_labels.get(signal, signal)

    if signal in motion_signals:
        if not motion_trace_added:
            ax.plot(signals.index, signals[signal], label=label, color=color)
            motion_trace_added = True
        else:
            ax.plot(signals.index, signals[signal] + motion_offset, label=f"{label}", color=color)
    else:
        ax.plot(signals.index, signals[signal], label=label, color='b')

    ax.set_ylabel(label)
    if signal in motion_signals:
        ax.legend(loc="upper right")

    if signal in y_limits:
        ax.set_ylim(y_limits[signal])

    row_idx += (1 if not (signal in motion_signals and motion_trace_added) else 0)

for ax in axes:
    for event_label, (color, alpha) in event_highlights.items():
        event_times = signals[signals["event"] == event_label].index
        if not event_times.empty:
            ax.axvspan(event_times[0], event_times[-1], color=color, alpha=alpha)

            ax.axvline(event_times[0], color="red", linestyle="--", linewidth=1)
            ax.axvline(event_times[-1], color="red", linestyle="--", linewidth=1)

            verbose_label = event_labels.get(event_label, event_label)
            ax.text(event_times[0], ax.get_ylim()[1], verbose_label, rotation=90,
                    verticalalignment="top", color="b", fontsize=12)



for ax in axes:
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

axes[-1].set_xlabel("Time")
fig.suptitle(f"Participant {participant_id} Physiological Signals", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.98])
plt.savefig(os.path.join(dataDirectory, "labeled_data.png"))
plt.show()

In [16]:
signals.head()
summary_stats = signals.describe()
summary_stats.to_csv('/home/art/SignalProcessing/Sessions/P009/summary.csv')

In [14]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_resampler import FigureWidgetResampler

# Load your data
signals = pd.read_pickle('/home/art/SignalProcessing/Sessions/P008/labeled_features.pkl')
participant_id = '8'

# Define signal lists and settings
signal_list = [
    'hrv_hf', 'hrv_mean_hr', 'hrv_lf_hf_ratio',
    'hrv_rmssd', 'Motion E4', 'Hexoskin Motion',
    'RSP_Rate', 'EDA_Tonic', 'EDA_Phasic'
]

signal_labels = {
    'hrv_hf': 'HRV High Frequency',
    'hrv_mean_hr': 'Mean Heart Rate (bpm)',
    'hrv_lf_hf_ratio': 'HRV LF/HF Ratio',
    'hrv_rmssd': 'HRV RMSSD (ms)',
    'Motion E4': 'E4 Motion Sensor',
    'Hexoskin Motion': 'Hexoskin Motion Sensor',
    'RSP_Rate': 'Respiratory Rate (bpm)',
    'EDA_Tonic': 'Electrodermal Activity (Tonic)',
    'EDA_Phasic': 'Electrodermal Activity (Phasic)'
}

event_colors = {
    "survey":      "#a6cee3",
    "cycle_cal":   "#1f78b4",
    "matb_train":  "#b2df8a",
    "cycle":       "#33a02c",
    "rest_1":      "#fb9a99",
    "baseline":    "#e31a1c",
    "trial_1":     "#fdbf6f",
    "rest_2":      "#ff7f00",
    "trial_2":     "#cab2d6",
    "recovery":    "#6a3d9a"
}

event_labels = {
    "survey": "Survey",
    "cycle_cal": "Calibration",
    "matb_train": "Training",
    "cycle": "Cycling",
    "rest_1": "Rest 1",
    "baseline": "Baseline",
    "trial_1": "Trial 1",
    "rest_2": "Rest 2",
    "trial_2": "Trial 2",
    "recovery": "Recovery"
}

# Set up subplots
num_signals = len(signal_list)
fig = make_subplots(rows=num_signals, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02, subplot_titles=[signal_labels[signal] for signal in signal_list if signal in signals.columns])

fig_resampler = FigureWidgetResampler(fig)

motion_offset = 0.7  # Offset for overlapping motion signals
motion_trace_added = False

# Add traces and event highlights
for i, signal in enumerate(signal_list, start=1):
    if signal in signals.columns:
        y_values = signals[signal]
        if signal in ['Motion E4', 'Hexoskin Motion'] and motion_trace_added:
            y_values += motion_offset  # Apply offset to the second motion signal
        trace = go.Scattergl(x=signals.index, y=y_values, mode='lines', name=signal_labels[signal])
        fig_resampler.add_trace(trace, hf_x=signals.index, hf_y=y_values, row=i, col=1)
        if signal in ['Motion E4', 'Hexoskin Motion']:
            motion_trace_added = True

        # Add event highlights with annotations for legend
        for event, color in event_colors.items():
            event_indices = signals[signals['event'] == event].index
            if not event_indices.empty:
                fig_resampler.add_vrect(
                    x0=event_indices[0], x1=event_indices[-1],
                    fillcolor=color, opacity=0.5,
                    layer="below", line_width=0,
                    row=i, col=1,
                    annotation_text=event_labels[event], annotation_position="top left"
                )

# Update layout
fig_resampler.update_layout(
    height=300 * num_signals,
    width=2000,
    title=f"Participant {participant_id} Physiological Signals",
    showlegend=True,
    template="plotly_white"
)

# Update axes and legends
fig_resampler.update_xaxes(tickformat="%H:%M:%S", tickangle=45)
fig_resampler.update_yaxes(fixedrange=True)

fig_resampler.show()



In [29]:
import neurokit2 as nk

analyzed = nk.eda_analyze(signals['EDA_Raw'].iloc[0], sampling_rate=256, method='auto')

UnboundLocalError: cannot access local variable 'features' where it is not associated with a value

In [5]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly_resampler import FigureResampler
from plotly_resampler.aggregation import NoGapHandler

app = dash.Dash(__name__)

signals = pd.read_pickle('/home/art/SignalProcessing/Sessions/P009/labeled_features.pkl')
participant_id = '9'


signal_list = [
    'ECG_Clean', 'Cleaned PPG', 'RSP_Clean', 'hrv_lf', 'hrv_hf', 'EDA_Clean',
    'hrv_rmssd', 'hrv_sdnn', 'Motion E4', 'Hexoskin Motion',
    'ECG_Rate', 'RSP_Rate', 'EDA_Tonic', 'EDA_Phasic'
]


signals_to_plot = [sig for sig in signals.columns if sig in signal_list]


motion_signals = ['Motion E4', 'Hexoskin Motion']

num_rows = len(signals_to_plot) - len(motion_signals) + 1
fig = FigureResampler(
    make_subplots(
        rows=num_rows, cols=1, shared_xaxes=True,
        specs=[[{"secondary_y": True}] if i == 0 else [{}] for i in range(num_rows)]
    ),
    default_n_shown_samples=30000,
    verbose=False
)

row_idx = 1
motion_trace_added = False

for signal in signals_to_plot:
    fig.add_trace(
        go.Scattergl(mode='lines', name=signal),
        hf_x=signals.index, hf_y=signals[signal],
        row=row_idx, col=1
    )
    if signal in motion_signals:
        if not motion_trace_added:
            motion_trace_added = True
        else:
            continue
    else:
        row_idx += 1

for event_label in signals['event'].unique():
    if event_label != "unknown":
        event_times = signals[signals['event'] == event_label].index
        for i in range(1, num_rows + 1):
            fig.add_vrect(x0=event_times[0], x1=event_times[-1], fillcolor="red", opacity=0.3, line_width=0, row=i, col=1)
            fig.add_annotation(
                x=event_times[0], y=0.95, text=event_label, showarrow=False,
                font=dict(color="blue", size=12),
                yshift=10, textangle=90, xshift=5,
                xref=f"x{i}", yref=f"y{i}"
            )



fig.update_layout(
    height=1200,
    title_text=f"Participant {participant_id} Physiological Signals",
    xaxis_title="Time",
    showlegend=True,
    autosize=True,
)

# Dash Layout
app.layout = html.Div([
    html.H1(f"Physiological Signal Dashboard - Participant {participant_id}"),
    dcc.Graph(figure=fig, id='signal-graph'),
])

if __name__ == '__main__':
    app.run_server(debug=True)